In [1]:
%load_ext autoreload
%autoreload 2

from src.robin.demand.entities import Demand

from pathlib import Path

In [2]:
path_config_demand = Path('../configs/ewgt23/demand_data_fuzzy.yml')

demand = Demand.from_yaml(path_config_demand)
demand

Demand(days=[Day(id=1, date=2023-09-06, demand_pattern=Monday-Thursday)])

In [3]:
passengers = demand.generate_passengers()
passengers

[Passenger(id=307, user_pattern=Tourist, market=60000 - 04040, arrival_day=2023-09-06, arrival_time=0.1492084671301228, purchase_day=14, service=None, service_departure_time=None, service_arrival_time=None, seat=None, ticket_price=None, utility=None,
 Passenger(id=349, user_pattern=Tourist, market=60000 - 04040, arrival_day=2023-09-06, arrival_time=14.663889894662566, purchase_day=14, service=None, service_departure_time=None, service_arrival_time=None, seat=None, ticket_price=None, utility=None,
 Passenger(id=408, user_pattern=Tourist, market=60000 - 04040, arrival_day=2023-09-06, arrival_time=23.340588826166304, purchase_day=14, service=None, service_departure_time=None, service_arrival_time=None, seat=None, ticket_price=None, utility=None,
 Passenger(id=499, user_pattern=Tourist, market=60000 - 04040, arrival_day=2023-09-06, arrival_time=12.055472350194234, purchase_day=14, service=None, service_departure_time=None, service_arrival_time=None, seat=None, ticket_price=None, utility=No

In [17]:
passengers[0].user_pattern.behaviour_variables.keys()

dict_keys(['origin', 'destination', 'departure_time', 'arrival_time', 'price', 'seat'])

In [53]:
import numpy as np

from typing import Union

from src.robin.demand.entities import Passenger

def get_constrain_value(passenger: Passenger,
                        variable_name: str,
    ) -> Union[float, np.NaN]:
    """
    Get the maximum value of a variable in the user pattern
    """
    max_value = np.NaN
    for rule in passenger.user_pattern.behaviour_rules:
        for proposition in rule.antecedent.proposiciones:
            if proposition.variable.name == variable_name:
               max_value = max(proposition.term.values)
    return max_value

get_constrain_value(passengers[0], 'origin')

20

In [5]:
import yaml

path_config_user_pattern = Path('../configs/ewgt23/demand_data_fuzzy.yml')

# Read yaml file
with open(path_config_user_pattern, 'r') as stream:
    demand_config = yaml.safe_load(stream)
    
demand_config

{'market': [{'id': 1,
   'departure_station': '60000',
   'departure_station_coords': [40.416775, -3.70379],
   'arrival_station': '04040',
   'arrival_station_coords': [41.648822, -0.889085]},
  {'id': 2,
   'departure_station': '04040',
   'departure_station_coords': [41.648822, -0.889085],
   'arrival_station': '71801',
   'arrival_station_coords': [41.385064, 2.173404]},
  {'id': 3,
   'departure_station': '60000',
   'departure_station_coords': [40.416775, -3.70379],
   'arrival_station': '71801',
   'arrival_station_coords': [41.385064, 2.173404]}],
 'userPattern': [{'id': 1,
   'name': 'Business',
   'rules': {'R0': 'IF (price is expensive) THEN 20.0',
    'R1': 'IF (origin is very_near) & (destination is very_near) THEN 16.0',
    'R2': 'IF (origin is very_near) & (destination is very_near) THEN 24.0',
    'R3': 'IF (origin is very_near) & (destination is very_near) & (departure_time is in_time) & (arrival_time is in_time) THEN 40.0'},
   'variables': [{'name': 'origin',
     '

In [6]:
from src.robin.demand.entities import UserPattern

user_pattern_data = demand_config['userPattern']
user_pattern_data

[{'id': 1,
  'name': 'Business',
  'rules': {'R0': 'IF (price is expensive) THEN 20.0',
   'R1': 'IF (origin is very_near) & (destination is very_near) THEN 16.0',
   'R2': 'IF (origin is very_near) & (destination is very_near) THEN 24.0',
   'R3': 'IF (origin is very_near) & (destination is very_near) & (departure_time is in_time) & (arrival_time is in_time) THEN 40.0'},
  'variables': [{'name': 'origin',
    'type': 'fuzzy',
    'support': [0, 100],
    'sets': ['very_near', 'mid_range', 'far', 'far_away'],
    'very_near': [0, 0, 10, 20],
    'mid_range': [10, 20, 50, 60],
    'far': [50, 60, 70, 80],
    'far_away': [70, 80, 100, 100]},
   {'name': 'destination',
    'type': 'fuzzy',
    'support': [0, 100],
    'sets': ['very_near', 'mid_range', 'far', 'far_away'],
    'very_near': [0, 0, 10, 20],
    'mid_range': [10, 20, 50, 60],
    'far': [50, 60, 70, 80],
    'far_away': [70, 80, 100, 100]},
   {'name': 'departure_time',
    'type': 'fuzzy',
    'support': [0, 100],
    'sets

In [4]:
user_patterns = {}
for user_pattern in demand_config['userPattern']:
    # Convert the forbidden departure hours into a tuple (begin, end)
    forbidden_departure_hours = tuple(user_pattern['forbidden_departure_hours'].values())
    user_pattern.pop('forbidden_departure_hours', None)

    # Convert the list of seats into a dictionary {id: utility}
    ids = []
    utilities = []
    for seat in user_pattern['seats']:
        ids.append(seat['id'])
        utilities.append(seat['utility'])
    seats = dict(zip(ids, utilities))
    user_pattern.pop('seats', None)

    user_patterns[user_pattern['id']] = UserPattern(
        forbidden_departure_hours=forbidden_departure_hours,
        seats=seats,
        **user_pattern
    )

user_patterns

{1: UserPattern(id=1, name=Business, arrival_time=<scipy.stats._continuous_distns.norm_gen object at 0x159e682d0>, arrival_time_kwargs={'loc': 8, 'scale': 1}, purchase_day=<scipy.stats._discrete_distns.randint_gen object at 0x127e02e50>, purchase_day_kwargs={'low': 2, 'high': 7}, forbidden_departure_hours=(9, 24), seats={1: 10, 2: 15, 3: 20}, penalty_arrival_time=<functools._lru_cache_wrapper object at 0x110eea8d0>, penalty_arrival_time_kwargs={'b_0': 0, 'b_1': 1}, penalty_departure_time=<functools._lru_cache_wrapper object at 0x110eea8d0>, penalty_departure_time_kwargs={'b_0': 0, 'b_1': 0.7}, penalty_cost=<functools._lru_cache_wrapper object at 0x110eea8d0>, penalty_cost_kwargs={'b_0': 0, 'b_1': 0.05}, penalty_travel_time=<functools._lru_cache_wrapper object at 0x110eea8d0>, penalty_travel_time_kwargs={'b_0': 0, 'b_1': 0.4}, error=<scipy.stats._continuous_distns.norm_gen object at 0x159e682d0>, error_kwargs={'loc': 2, 'scale': 1}),
 2: UserPattern(id=2, name=Student, arrival_time=<sci

In [8]:
user_patterns[1]

UserPattern(id=1, name=Business, behaviour_rules=[<src.robin.decision_model.fuzzy_rule.TSKRule object at 0x15c09c310>, <src.robin.decision_model.fuzzy_rule.TSKRule object at 0x15aa79f50>, <src.robin.decision_model.fuzzy_rule.TSKRule object at 0x15c07a490>], behaviour_variables={'seat': {'name': 'seat', 'type': 'fuzzy', 'position': 0, 'support': [0, 100], 'sets': {'premium': <src.robin.decision_model.terms.MembershipFS object at 0x15ac38210>, 'standard': <src.robin.decision_model.terms.MembershipFS object at 0x122252a10>, 'tourist': <src.robin.decision_model.terms.MembershipFS object at 0x15abdbf90>}}, 'service_departure_time': {'name': 'service_departure_time', 'type': 'fuzzy', 'position': 1, 'support': [0, 100], 'sets': {'in_time': <src.robin.decision_model.terms.MembershipFS object at 0x15abdbd10>, 'close': <src.robin.decision_model.terms.MembershipFS object at 0x15abdba10>, 'far': <src.robin.decision_model.terms.MembershipFS object at 0x15abdb6d0>}}, 'service_arrival_time': {'name':